# Biostat 257 Homework 4

**Due May 20 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.7.2
Commit bf53498635 (2022-02-06 15:21 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-4260U CPU @ 1.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, haswell)


We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

## Q1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

#### Solution

$\mathbf{P} = [p_{ij}]$ where $p_{ij}$ is the probability that a surfer links to page j next step given that surfer is currently at page i. Therefore,

$$p_{ij} = \begin{cases}
      p * \frac{a_{ij}}{r_i} + \frac{1-p}{n} & \text{if $r_i > 0$}\\
      1/n & \text{if $r_i = 0$}
      \end{cases} $$

Separating this into a sparse matrix plus rank 1 matrix, we get $\mathbf{P} = \mathbf{D} \mathbf{A} + \mathbf{c} \mathbf{1}^T$. $\mathbf{D}$ is a diagonal matrix of $d_i$ where

$$d_i = \begin{cases}
      \frac{p}{r_i} & \text{if $r_i > 0$}\\
      0 & \text{if $r_i = 0$}
      \end{cases} 
$$

$\mathbf{A}$ is a sparse matrix with the elements described above. $\mathbf{c}$ is a vector of $c_i$ where

$$c_i = \begin{cases}
      \frac{1-p}{n} & \text{if $r_i > 0$}\\
      \frac{1}{n} & \text{if $r_i = 0$}
      \end{cases} 
$$

and $\mathbf{1}^T$ is a nxn vector of ones.

## Q2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

## Q3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [13]:
# import Pkg; Pkg.add("MatrixDepot")
# import Pkg; Pkg.add("UnicodePlots")

In [1]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

┌ Info: verify download of index files...
└ @ MatrixDepot /Users/amisheth/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:139
┌ Info: reading database
└ @ MatrixDepot /Users/amisheth/.julia/packages/MatrixDepot/GEDc3/src/download.jl:23
┌ Info: adding metadata...
└ @ MatrixDepot /Users/amisheth/.julia/packages/MatrixDepot/GEDc3/src/download.jl:67
┌ Info: adding svd data...
└ @ MatrixDepot /Users/amisheth/.julia/packages/MatrixDepot/GEDc3/src/download.jl:69
┌ Info: writing database
└ @ MatrixDepot /Users/amisheth/.julia/packages/MatrixDepot/GEDc3/src/download.jl:74
┌ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index
└ @ MatrixDepot /Users/amisheth/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:141


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [2]:
# connectivity matrix
A = md.A

916428×916428 SparseArrays.SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:
* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  
* number of web pages
* number of edges (web links). 
* number of dangling nodes (pages with no out links)
* histogram of in-degrees  
* list the top 20 pages with the largest in-degrees?  
* histogram of out-degrees
* which the top 20 pages with the largest out-degrees?
* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

#### Summary Statistics Solutions

1. How much memory does `A` take? If converted to a `Matrix{Float64}`, how much memory will it take?

In [6]:
Base.summarysize(A) # storing non-zero float entries and their position

53276943

`A` takes 53.28 megabytes of memory. If converted to `Matrix{Float64}`, `A` would take $n^2$ (number of double-precision elements in `A`) * $8$ bytes (memory per `Float64`) = $6.72$ TB.

2. Number of web pages

In [8]:
size(A, 1) # node different than in description?

916428

3. Number of edges (web links)

In [9]:
count(!iszero, A)

5105039

4. Number of dangling nodes (pages with no links)

In [10]:
outdegree = sum(A, dims = 2) # row sum
count(iszero, outdegree) # counting non-zero elements

176974

5. Histogram of in-degrees

In [61]:
using UnicodePlots

indegree = sum(A, dims = 1) # column sum
histogram(indegree)

                    ┌                                        ┐ 
   [   0.0,  500.0) ┤████████████████████████████████  916114  
   [ 500.0, 1000.0) ┤▏ 180                                     
   [1000.0, 1500.0) ┤▏ 32                                      
   [1500.0, 2000.0) ┤▏ 20                                      
   [2000.0, 2500.0) ┤▏ 16                                      
   [2500.0, 3000.0) ┤▏ 20                                      
   [3000.0, 3500.0) ┤▏ 18                                      
   [3500.0, 4000.0) ┤▏ 10                                      
   [4000.0, 4500.0) ┤▏ 9                                       
   [4500.0, 5000.0) ┤▏ 4                                       
   [5000.0, 5500.0) ┤▏ 4                                       
   [5500.0, 6000.0) ┤  0                                       
   [6000.0, 6500.0) ┤▏ 1                                       
                    └                                        ┘ 
                                     Fre

6. List the top 20 pages with largest in-degrees

In [62]:
# sortperm gives indices of the sorted vector
sortperm(indegree[1, :], rev = true)[1:20] 

20-element Vector{Int64}:
 537040
 597622
 504141
 751385
  32164
 885606
 163076
 819224
 605857
 828964
 551830
  41910
 558792
 459075
 407611
 213433
 765335
 384667
 173977
 687326

7. Histogram of out-degrees

In [15]:
histogram(outdegree)

                  ┌                                        ┐ 
   [  0.0,  20.0) ┤████████████████████████████████  891798  
   [ 20.0,  40.0) ┤▊ 22628                                   
   [ 40.0,  60.0) ┤▏ 1329                                    
   [ 60.0,  80.0) ┤▏ 371                                     
   [ 80.0, 100.0) ┤▏ 124                                     
   [100.0, 120.0) ┤▏ 86                                      
   [120.0, 140.0) ┤▏ 29                                      
   [140.0, 160.0) ┤▏ 15                                      
   [160.0, 180.0) ┤▏ 15                                      
   [180.0, 200.0) ┤▏ 8                                       
   [200.0, 220.0) ┤▏ 7                                       
   [220.0, 240.0) ┤▏ 3                                       
   [240.0, 260.0) ┤▏ 8                                       
   [260.0, 280.0) ┤▏ 3                                       
   [280.0, 300.0) ┤  0                                       
   [300.

8. List the top 20 pages with the largest out-degrees

In [16]:
sortperm(outdegree[:, 1], rev = true)[1:20]

20-element Vector{Int64}:
 506743
 203749
 305230
 768092
 808644
 412411
 600480
 376429
 156951
 885729
 667585
 685696
 282141
 598189
 579315
 411594
 321092
 838279
 302734
 915274

9. Visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`

In [17]:
spy(A)

          ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Sparsity Pattern⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    
          ┌──────────────────────────────────────────┐    
        1 │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ > 0
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ < 0
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│  

In [18]:
spy(A[1:10000, 1:10000])

         ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Sparsity Pattern⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    
         ┌──────────────────────────────────────────┐    
       1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
         │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
         │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
         │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
         │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
         │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
         │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
         │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
         │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
         │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
         │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
         │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
         │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
         │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
         │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
         │⠈⠀⠀⠄

## Q4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

#### Solution 

1. Memory usage

`A` is over written by a L - lower triangular matrix and U - upper triangular matrix, which means that both L and U will have $n(n+1)/2$ unique elements to store. This results in a total of $n^2 + n$ elements of `Float64`, which estimates to $6.72$ terabytes.

2. Time

LU decomposition takes $\frac{2}{3} n^3 + 2n^2$ flops, so given our data this would require $\frac{2}{3} (916428)^3 + 2(916428)^2 = 5.131 x 10^{17}$ flops

In [7]:
using LinearAlgebra

5.131e17 / LinearAlgebra.peakflops()

1.4532095710168751e7

LU decomposition would take about $1.45 x 10^7$ seconds, which is about 168 days. 

## Q5 (75 pts) Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [28]:
# import Pkg; Pkg.add("Revise")

In [40]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    d         :: Vector{T} # working arrays
    c         :: Vector{T}
    store_n   :: Vector{T}
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    outdegree = sum(A, dims = 2)[:, 1] # vector of row sums
    d = zeros(n) # diagonal elements to be multiplied by A (nx1)
    c = zeros(n) # vector of constants (nx1)
    for i = 1:n
        if outdegree[i] > 0
            d[i] = telep / outdegree[i] 
            c[i] = (1 - telep) / n 
        else 
            d[i] = 0
            c[i] = 1 / n
        end
    end
    store_n = Vector{T}(undef, n) # stores nx1 vector
    PageRankImPt(A, telep, d, c, store_n)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false 
Base.size(M::PageRankImPt) = size(M.A) # creates matrix M (nxn) 

# implement this function for evaluating M[i, j]
# M = I - P' = I - A'D' - 1c'
Base.getindex(M::PageRankImPt, i, j) = (i == j) - M.d[j] * M.A[j, i] - M.c[j] 

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}
        ) where T <: AbstractFloat
    
    # implementing mul!(out, M, v)
    
    M.store_n .= M.d .* v # Dv (nx1), element-wise multiplication
    ctv = dot(M.c, v) # c'v (constant)
    
    # out = A'D'v (nx1)
    mul!(out, transpose(M.A), M.store_n)
    
    # Mv = v - A'D'v - 1(c'v)
    out .= v .- out .- ctv
    
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}
        ) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    
    M = Mt.parent # M is now pointing back to original M
    
    # implementing mul!(out, transpose(M), v)
    
    sumv = sum(v) # 1'v (constant)
    
    # out = Av (nx1)
    mul!(out, M.A, v)
    
    # M'v = v - DAv - c1'v
    out .= v .- M.d .* out .- sumv .* M.c
    
    return out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [5]:
# import Pkg; Pkg.add("CodecZlib")

In [4]:
using CodecZlib, DelimitedFiles

isfile("pgrksol.csv.gz") || download("https://raw.githubusercontent.com/ucla-biostat-257/2022spring/master/hw/hw4/pgrksol.csv.gz")
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [41]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

#@assert transpose(M) * ones(n) ≈ zeros(n)
@assert norm(transpose(M) * ones(n)) < 1e-12

In [42]:
# @assert M * xsol ≈ zeros(n)
@assert norm(M * xsol) < 1e-12

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate little memory.

In [43]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup = (fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 77 samples with 1 evaluation.
 Range (min … max):  59.841 ms … 78.869 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     61.970 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   63.851 ms ±  4.641 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▃█   ▂                                                    
  ▆█▃██▇█▇██▅▁▃▅▁▁▇▁▁▁▅▃▃▅▁▁▃▁▁▁▁▁▁▁▃▁▁▁▁▃▃▁▁▁▁▁▁▁▃▃▁▁▁▁▁▃▃▁▅ ▁
  59.8 ms         Histogram: frequency by time          77 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [44]:
bm_mtv = 
@benchmark mul!($out, $(transpose(M)), $v) setup = (fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 51 samples with 1 evaluation.
 Range (min … max):  87.634 ms … 170.272 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     91.916 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   97.279 ms ±  17.015 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▇   ▂                                                        
  ██▆▇▆█▆▆▁▃▃▁▁▁▁▁▁▁▃▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▁
  87.6 ms         Histogram: frequency by time          165 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [45]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

**Hint**: My median run times are 30-40 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [15]:
# import Pkg; Pkg.add("KrylovKit")

In [46]:
using KrylovKit

# normalize in-degrees to be the start point
x0 = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1)

# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, 
    issymmetric = false, isposdef = false)

(value = ([3.3783428221986054e-5, 2.0710155392544003e-6, 3.6630659852478028e-6, 7.527510785625115e-7, 8.633285997186966e-7, 1.7694182527430189e-6, 2.4312303829196376e-7, 6.368417180766123e-7, 4.744973703764497e-7, 2.689548611104835e-7  …  3.1679469739868288e-6, 9.726880410259511e-7, 6.588614478562195e-7, 7.737011774737646e-7, 2.4312303829196376e-7, 1.6219204214293997e-6, 3.9121300606432984e-7, 2.4312303829196376e-7, 7.2960338313468936e-6, 6.330939996697664e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821151654716082e-13,).
), time = 13.029900215, bytes = 1008307445, gctime = 0.144321126, gcstats = Base.GC_Diff(1008307445, 137, 0, 64151, 4, 0, 144321126, 3, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [47]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [48]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 7-8 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [49]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)

# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, 
    issymmetric = false)

(value = (ComplexF64[-3.653836029264281e-14 + 0.0im], Vector{ComplexF64}[[0.005635826953807308 + 0.0im, 0.00034549143807828384 + 0.0im, 0.000611080849412529 + 0.0im, 0.00012557561626036273 + 0.0im, 0.00014402240532804286 + 0.0im, 0.0002951783050398634 + 0.0im, 4.055832828678566e-5 + 0.0im, 0.00010623935784852789 + 0.0im, 7.915671116196516e-5 + 0.0im, 4.4867650667256554e-5 + 0.0im  …  0.0005284839899796966 + 0.0im, 0.0001622659914798883 + 0.0im, 0.00010991273837610668 + 0.0im, 0.00012907055855752622 + 0.0im, 4.055832828678567e-5 + 0.0im, 0.00027057238743205243 + 0.0im, 6.526302748334149e-5 + 0.0im, 4.055832828678567e-5 + 0.0im, 0.001217140660132936 + 0.0im, 0.00010561415510700478 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.575262040976209e-14,).
), time = 24.16494643, bytes = 1580664720, gctime = 0.774685695, gcstats = Base.GC_Diff(1580664720, 213, 0, 66863, 20, 0, 774685695, 4, 2))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [50]:
@assert abs(Real(vals[1])) < 1e-8

In [51]:
x_arnoldi = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [52]:
clamp(20 / time_arnoldi * 20, 0, 20)

16.552902410055125

In [53]:
time_arnoldi

24.16494643

**Hint**: My runtime is about 11-12 seconds.

**Note:** My computer is slower than Dr. Zhou's, he said it is okay

## Q6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

In [71]:
sort(x_arnoldi, rev = true)[1:20] # page-ranks for top 20 pages

20-element Vector{Float64}:
 0.0009145812114518193
 0.0009120131809985913
 0.0008950559016075249
 0.0008899344804414284
 0.000779103179016378
 0.0007575423485846715
 0.0007177642925774723
 0.0007108483954459588
 0.0007055182681589446
 0.0007021658710708819
 0.0006950751256446825
 0.000676227602573153
 0.0006546558408067942
 0.0006168480316346435
 0.0006146220814775504
 0.0006031151911310471
 0.0005926642909131656
 0.0005894474426054643
 0.0005812660189536186
 0.000576518974454283

In [59]:
sortperm(x_arnoldi, rev = true)[1:20]

20-element Vector{Int64}:
 597622
  41910
 163076
 537040
 384667
 504141
 486981
 605857
  32164
 558792
 551830
 765335
 751385
 425771
 908352
 173977
   7315
 213433
 885606
 819224

In [63]:
sortperm(indegree[1, :], rev = true)[1:20] 

20-element Vector{Int64}:
 537040
 597622
 504141
 751385
  32164
 885606
 163076
 819224
 605857
 828964
 551830
  41910
 558792
 459075
 407611
 213433
 765335
 384667
 173977
 687326

## Q7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.